In [1]:
# in a terminal run
# > USER_PATH=/mnt/d/workspace/clotho/userdata/ make run-redis NS=train
# > USER_PATH=/mnt/d/workspace/clotho/userdata/ make run-redis NS=test
# to allow access to the train and test namespaces

In [2]:
import os
import sys
import json
import pandas as pd
import numpy as np
from typing import Literal, TypedDict

In [3]:
sys.path.append("..")
os.environ["USER_PATH"] = "/mnt/d/workspace/clotho/userdata/"
MODEL_OUTPUT_BASE = "/mnt/d/workspace/clotho/notebooks"
MODEL_OUTPUT_CP = os.path.join(MODEL_OUTPUT_BASE, "checkpoints")

In [4]:
from misc.redis import set_redis_slow_mode
from misc.util import highest_number
from misc.io import open_write
from model.datagenerator import create_train_test
from model.transformer_embed import (
    get_epoch_and_load,
    limit_epoch_data,
    limit_epoch_data,
    get_model_filename,
)
from system.namespace.store import get_namespace

In [5]:
import torch

is_cuda = torch.cuda.is_available()
is_cuda

True

In [6]:
set_redis_slow_mode("never")
ns_test = get_namespace("test")
ns_train = get_namespace("train")
now = pd.Timestamp("2022-12-17", tz="UTC")
train_plan = [
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 50,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 50,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
     {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 15,
        "last_epoch": None,
        "weight": 50,
    }
]
eval_plan = [
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
]
ttgen = create_train_test(
    train_ns=ns_train,
    train_validation_ns=ns_train,
    test_ns=ns_test,
    test_validation_ns=ns_test,
    train_learning_plan=train_plan,
    train_val_learning_plan=eval_plan,
    test_learning_plan=eval_plan,
    test_val_learning_plan=eval_plan,
    batch_size=4 if is_cuda else 8,
    epoch_batches=5000 if is_cuda else 500,
    train_val_size=10000 if is_cuda else 1000,
    test_size=10000 if is_cuda else 1000,
    test_val_size=10000 if is_cuda else 1000,
    compute_batch_size=100 if is_cuda else 100,
    now=now)

In [7]:
import torch.nn as nn
from torch.optim import AdamW
from transformers import DistilBertTokenizer, DistilBertModel

In [8]:
device = torch.device("cuda") if is_cuda else torch.device("cpu")
device

device(type='cuda')

In [9]:
ProviderRole = Literal["child", "parent"]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
EMBED_SIZE = 768

TokenizedInput = TypedDict('TokenizedInput', {
    "input_ids": torch.Tensor,
    "attention_mask": torch.Tensor,
})


AggType = Literal["cls", "mean"]
AGG_CLS: AggType = "cls"
AGG_MEAN: AggType = "mean"


def tokens(texts: list[str]) -> TokenizedInput:
    res = tokenizer(texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    return {k: v.to(device) for k, v in res.items()}


class Noise(nn.Module):
    def __init__(self, std: float = 1.0, p: float = 0.5) -> None:
        super().__init__()
        self._std = std
        self._p = p
        self._dhold = nn.Parameter(torch.Tensor([0.0]), requires_grad=False)

    def set_std(self, std: float) -> None:
        self._std = std

    def get_std(self) -> float:
        return self._std

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training:
            return x
        prob = torch.rand(size=x.shape, device=self._dhold.device) < self._p
        gauss = torch.normal(
            mean=0.0, std=self._std, size=x.shape, device=self._dhold.device)
        return x + prob * gauss


class Model(nn.Module):
    def __init__(self, version: int) -> None:
        super().__init__()
        self._bert_parent = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        self._bert_child = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        if version in (1, 3, 4, 6):
            self._pdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
            self._cdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
        else:
            self._pdense = None
            self._cdense = None
        if version < 4 or version > 5:
            self._noise = None
        else:
            self._noise = Noise(std=1.0, p=0.2)
        if version < 2 or version > 4:
            self._cos = None
        else:
            self._cos = torch.nn.CosineSimilarity()
        if version < 6:
            self._agg = AGG_CLS
        else:
            self._agg = AGG_MEAN
        self._version = version

    def set_epoch(self, epoch: int) -> None:
        noise = self._noise
        if noise is not None:
            noise.set_std(1 / (1.2 ** epoch))

    def get_version(self) -> int:
        return self._version
    
    def get_agg(self, lhs: torch.Tensor) -> torch.Tensor:
        if self._agg == AGG_CLS:
            return lhs[:, 0]
        if self._agg == AGG_MEAN:
            return torch.mean(lhs, dim=1)
        raise ValueError(f"unknown aggregation: {self._agg}")

    def get_parent_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_parent = self._bert_parent(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_parent.last_hidden_state)
        if self._pdense is not None:
            out = self._pdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def get_child_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_child = self._bert_child(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_child.last_hidden_state)
        if self._cdense is not None:
            out = self._cdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def forward(self, x: dict[ProviderRole, TokenizedInput]) -> torch.Tensor:
        parent_cls = self.get_parent_embed(
            input_ids=x["parent"]["input_ids"],
            attention_mask=x["parent"]["attention_mask"])
        child_cls = self.get_child_embed(
            input_ids=x["child"]["input_ids"],
            attention_mask=x["child"]["attention_mask"])
        if self._cos is not None:
            return self._cos(parent_cls, child_cls).reshape([-1, 1])
        batch_size = parent_cls.shape[0]
        return torch.bmm(
            parent_cls.reshape([batch_size, 1, -1]),
            child_cls.reshape([batch_size, -1, 1])).reshape([-1, 1])


class TrainingHarness(nn.Module):
    def __init__(self, model: Model) -> None:
        super().__init__()
        self._model = model
        self._softmax = nn.Softmax(dim=1)
        self._loss = nn.BCELoss()

    def get_version(self) -> int:
        return self._model.get_version()

    def forward(
            self,
            left: TokenizedInput,
            right: TokenizedInput,
            labels: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        out_left = self._model(left)
        out_right = self._model(right)
        preds = self._softmax(torch.hstack((out_left, out_right)))
        return preds, self._loss(preds, labels)

In [10]:
FORCE_RESTART = False

In [11]:
from transformers import get_scheduler
# from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import evaluate
import time


def compute(harness, df):
    plefts = tokens(df["parent_left"])
    clefts = tokens(df["child_left"])
    prights = tokens(df["parent_right"])
    crights = tokens(df["child_right"])
    labels = torch.tensor(
        [~df["correct_is_right"], df["correct_is_right"]],
        dtype=torch.float32).T.to(device)
    return harness(
        left={"parent": plefts, "child": clefts},
        right={"parent": prights, "child": crights},
        labels=labels)


def run_training(num_epochs, version):
    model = Model(version=version)
    model.to(device)
    harness = TrainingHarness(model)
    harness.to(device)

    mprev, epoch_offset = get_epoch_and_load(
        harness,
        MODEL_OUTPUT_CP,
        ftype="harness",
        is_cuda=is_cuda,
        device=device,
        force_restart=FORCE_RESTART)

    optimizer = AdamW(harness.parameters(), lr=5e-5)
    print(mprev, epoch_offset)
    
    num_epochs -= epoch_offset
    if num_epochs <= 0:
        print("already computed all epochs. nothing to do!")
        return model, harness, optimizer
    
    num_training_steps = num_epochs * ttgen.get_epoch_train_size()
    warmup = 10000 if is_cuda else 10
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=warmup,
        num_training_steps=num_training_steps - warmup)
    ttgen.set_epoch(epoch_offset)

    for _ in range(num_epochs):
        epoch = ttgen.get_epoch()
        print(f"epoch {epoch} version: {harness.get_version()}")
        real_time = time.monotonic()

        model.train()
        harness.train()
        model.set_epoch(epoch)
        metric_train = evaluate.load("accuracy")
        train_loss = []
        first = True
        with tqdm(desc="train", total=ttgen.get_epoch_train_size()) as progress_bar:
            for train_df in ttgen.train_dfs():
                preds, loss = compute(harness, train_df)
                train_loss.append(loss.item())
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(train_df.shape[0])

                predictions = torch.argmax(preds, dim=-1)
                metric_train.add_batch(
                    predictions=predictions,
                    references=train_df["correct_is_right"].astype(int))
                if first:
                    # display(train_df)
                    first = False

        model_fname = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="harness",
            epoch=epoch)
        torch.save(harness.state_dict(), model_fname)

        model.eval()
        harness.eval()
        with torch.no_grad():
            metric_val_train = evaluate.load("accuracy")
            train_val_loss = []
            with tqdm(desc="train val", total=ttgen.get_epoch_train_validation_size()) as progress_bar:
                for train_validation_df in ttgen.train_validation_dfs():
                    preds, loss = compute(harness, train_validation_df)
                    train_val_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_val_train.add_batch(
                        predictions=predictions,
                        references=train_validation_df["correct_is_right"].astype(int))
                    progress_bar.update(train_validation_df.shape[0])

            metric_test = evaluate.load("accuracy")
            test_loss = []
            with tqdm(desc="test", total=ttgen.get_epoch_test_size()) as progress_bar:
                for test_df in ttgen.test_dfs():
                    preds, loss = compute(harness, test_df)
                    test_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_test.add_batch(
                        predictions=predictions,
                        references=test_df["correct_is_right"].astype(int))
                    progress_bar.update(test_df.shape[0])
            stats = {
                "epoch": int(epoch),
                "train_acc": float(metric_train.compute()['accuracy']),
                "train_loss": float(np.mean(train_loss)),
                "train_val_acc": float(metric_val_train.compute()['accuracy']),
                "train_val_loss": float(np.mean(train_val_loss)),
                "test_acc": float(metric_test.compute()['accuracy']),
                "test_loss": float(np.mean(test_loss)),
                "time": 0.0,
                "version": harness.get_version(),
                "fname": model_fname,
            }

        print(f"train: {stats['train_acc']} loss: {stats['train_loss']}")
        print(f"train val: {stats['train_val_acc']} loss: {stats['train_val_loss']}")
        print(f"test: {stats['test_acc']} loss: {stats['test_loss']}")
        ttgen.advance_epoch()
        stats["time"] = float((time.monotonic() - real_time) / 60.0)
        print(f"epoch time: {stats['time']:.2f}min")
        stats_fn = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            epoch=epoch,
            ext=".json")
        with open_write(stats_fn, text=True) as fout:
            print(json.dumps(stats, indent=2, sort_keys=True), file=fout)
            
        limit_epoch_data(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            ext=".json",
            count=5)
    return model, harness, optimizer

In [12]:
model_final, harness_final, optimizer_final = run_training(120, 7)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- T

('harness_v7_lg_74.pkl', 74) 75
epoch 75 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

train: 0.88495 loss: 0.2377157407395076
train val: 0.7876 loss: 0.41121565336245985
test: 0.6055 loss: 0.6637235795557499
epoch time: 38.51min
removing /mnt/d/workspace/clotho/notebooks/checkpoints/stats_v7_lg_109.json
removing /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v7_lg_109.pkl
epoch 119 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: 0.89205 loss: 0.22627422869708816
train val: 0.7835 loss: 0.4157945149488747
test: 0.625 loss: 0.6556196522563696
epoch time: 35.52min
removing /mnt/d/workspace/clotho/notebooks/checkpoints/stats_v7_lg_115.json
removing /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v7_lg_115.pkl


In [13]:
torch.save(model_final.state_dict(), get_model_filename(
    harness_final,
    MODEL_OUTPUT_BASE,
    is_cuda=is_cuda,
    ftype="model",
    epoch=None))
torch.save(harness_final.state_dict(), get_model_filename(
    harness_final,
    MODEL_OUTPUT_BASE,
    is_cuda=is_cuda,
    ftype="harness",
    epoch=None))
torch.save(optimizer_final.state_dict(), get_model_filename(
    harness_final,
    MODEL_OUTPUT_BASE,
    is_cuda=is_cuda,
    ftype="optimizer",
    epoch=None))

In [14]:
ttgen.reset()
model_final.eval()
harness_final.eval()
dfs = []
with torch.no_grad():
    metric_val_test = evaluate.load("accuracy")
    test_val_loss = []
    with tqdm(desc="test val", total=ttgen.get_epoch_test_validation_size()) as progress_bar:
        for test_val_df in ttgen.test_validation_dfs():
            preds, loss = compute(harness_final, test_val_df)
            test_val_loss.append(loss.item())
            predictions = torch.argmax(preds, dim=-1)
            metric_val_test.add_batch(
                predictions=predictions,
                references=test_val_df["correct_is_right"].astype(int))
            cur_df = test_val_df.copy()
            cur_df["logit_left"] = preds[:, 0].cpu()
            cur_df["logit_right"] = preds[:, 1].cpu()
            cur_df["preds"] = predictions.cpu()
            cur_df["truth"] = test_val_df["correct_is_right"].astype(int)
            dfs.append(cur_df)
            progress_bar.update(test_val_df.shape[0])
print(f"test val: {metric_val_test.compute()} loss: {np.mean(test_val_loss)}")
validation_df = pd.concat(dfs)

test val:   0%|          | 0/10000 [00:00<?, ?it/s]

test val: {'accuracy': 0.6303} loss: 0.6586172372460365


In [15]:
validation_df.to_csv(get_model_filename(
    harness_final,
    MODEL_OUTPUT_BASE,
    is_cuda=is_cuda,
    ftype="validation",
    epoch=None,
    ext=".csv"))

In [16]:
validation_df[validation_df["preds"] == validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
0,*valid--!copy;(mtl:20);(sw);(st),I once knew this family where the dad had to w...,What blows my mind about this the most is this...,What blows my mind about this the most is this...,I once knew this family where the dad had to w...,1.000000,1.645811e-38,False,0.704600,0.295400,0,0
1,random--valid;(sw);(st),I'll take the roast. That was hilarious,"Take care of your finances, your teeth, your b...",Would you be in favor of removing “One Nation ...,You don’t need to remove “One Nation.” Removin...,0.119203,8.807971e-01,True,0.228511,0.771489,1,1
2,random--valid;(sw);(st),Totally would. I think the more exposure you h...,No clue how i didnt see it on the first couple...,What’s something weird that porn has normalised?,Women hissing,0.047426,9.525741e-01,True,0.251375,0.748625,1,1
3,random--valid;(sw);(st),I really enjoyed Malcolm in the Middle as a ki...,Probably a young Steve buscemi at best.,"I’m not sure what this is called, but I have a...",I don't put the blame for bad designs like Poc...,0.047426,9.525741e-01,True,0.371450,0.628550,1,1
0,random--valid;(sw);(st),I edited it almost instantly,Because of those teachers in highschool my tea...,What dark secret are you hiding from everyone?,I won’t go on a throw away account. I became e...,0.047426,9.525741e-01,True,0.283972,0.716028,1,1


In [17]:
validation_df[validation_df["preds"] != validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
3,*valid--random;(sw);(st),If I could have guaranteed my daughter would i...,Dude I thought of the exact same thing! My dau...,so then whats the difference between OP (and g...,Only if it comes with enforcing not driving li...,1.000000,7.753691e-96,False,0.363922,0.636078,1,0
0,*valid--random;(mtl:20);(sw);(st),If you could rename America what would it be c...,Just name it all Florida,Cargo cults exist mostly because of the societ...,When your stomachs trap air then make a fart n...,1.000000,0.000000e+00,False,0.465955,0.534045,1,0
2,*valid--random;(sw);(st),I followed all the rules like a good little bo...,I think that a big part of it...,call her mayonnaise and tell her that if she f...,We'll never know until we get there. You've go...,0.880797,1.192029e-01,False,0.331302,0.668698,1,0
3,*valid--random;(sw);(st),Nice scumbag defence. Are you their legal coun...,Where do you see me defending any of these peo...,Girlfriend of 10 years convinced me to get her...,"No problem, bud! 👍",0.997527,2.472623e-03,False,0.485748,0.514252,1,0
2,random--valid;(sw);(st),"exactly!! and even if they get a F, you should...",It was worse… much much worse. Open racism and...,"After reading the comments, TIL a lot of peopl...","And also that americans call it ""tamale"" inste...",0.401312,5.986877e-01,True,0.530227,0.469773,0,1


In [18]:
ttgen.reset()
model_final.eval()
harness_final.eval()
with torch.no_grad():
    count = 0
    for test_val_df in ttgen.test_validation_dfs():
        plefts = tokens(test_val_df["parent_left"])
        clefts = tokens(test_val_df["child_left"])
        prights = tokens(test_val_df["parent_right"])
        crights = tokens(test_val_df["child_right"])
        display(model_final.get_child_embed(
            clefts["input_ids"],
            clefts["attention_mask"]).cpu().numpy())
        display(model_final.get_child_embed(
            crights["input_ids"],
            crights["attention_mask"]).cpu().numpy())
        count += 1
        if count >= 5:
            break

array([[-0.06550326,  0.02709419,  0.21112214, ...,  0.11420967,
        -0.38496572, -0.05269245],
       [-0.06571657,  0.02793262,  0.2312173 , ...,  0.11194899,
        -0.38510418, -0.05252533],
       [-0.06566565,  0.02773809,  0.2264941 , ...,  0.11248553,
        -0.38508695, -0.05256777],
       [-0.06568879,  0.02782691,  0.22864635, ...,  0.11224143,
        -0.38509583, -0.05254867]], dtype=float32)

array([[-0.06535928,  0.02648313,  0.19693018, ...,  0.11577048,
        -0.38476798, -0.05278942],
       [-0.06574368,  0.02803508,  0.23371796, ...,  0.11166368,
        -0.38510934, -0.05250207],
       [-0.0657125 ,  0.02791727,  0.23084284, ...,  0.11199162,
        -0.3851029 , -0.05252872],
       [-0.06563538,  0.02762094,  0.22366752, ...,  0.11280513,
        -0.38507187, -0.05259226]], dtype=float32)

array([[-0.0656829 ,  0.02780432,  0.22809815, ...,  0.11230366,
        -0.3850937 , -0.05255355],
       [-0.06569798,  0.02786201,  0.2294989 , ...,  0.11214454,
        -0.3850989 , -0.05254098],
       [-0.06571472,  0.0279257 ,  0.23104848, ...,  0.11196826,
        -0.3851037 , -0.05252688],
       [-0.06575096,  0.02806242,  0.23438704, ...,  0.1115872 ,
        -0.3851104 , -0.05249573]], dtype=float32)

array([[-0.06533301,  0.02636671,  0.19427145, ...,  0.11605955,
        -0.38472182, -0.05280573],
       [-0.06574761,  0.02804977,  0.23407796, ...,  0.11162256,
        -0.3851099 , -0.05249863],
       [-0.06567356,  0.02776845,  0.22722957, ...,  0.11240222,
        -0.38509   , -0.05256129],
       [-0.06568024,  0.02779406,  0.22784998, ...,  0.11233185,
        -0.38509274, -0.0525558 ]], dtype=float32)

array([[-0.06561287,  0.02753301,  0.22155438, ...,  0.11304329,
        -0.3850586 , -0.05261013],
       [-0.06574942,  0.02805658,  0.23424454, ...,  0.11160348,
        -0.3851102 , -0.05249709],
       [-0.06573675,  0.02800887,  0.2330777 , ...,  0.11173683,
        -0.38510838, -0.05250804],
       [-0.06563716,  0.02762791,  0.22383493, ...,  0.11278623,
        -0.38507292, -0.05259083]], dtype=float32)

array([[-0.06572815,  0.02797643,  0.23228577, ...,  0.11182722,
        -0.38510668, -0.05251546],
       [-0.06568072,  0.02779597,  0.22789657, ...,  0.11232647,
        -0.38509297, -0.05255538],
       [-0.06574763,  0.02804986,  0.23407973, ...,  0.11162236,
        -0.38511008, -0.05249863],
       [-0.06575137,  0.0280639 ,  0.23442353, ...,  0.11158305,
        -0.3851104 , -0.05249535]], dtype=float32)

array([[-0.0656904 ,  0.02783306,  0.22879606, ...,  0.11222443,
        -0.3850964 , -0.05254735],
       [-0.06578612,  0.02819377,  0.23761076, ...,  0.11121783,
        -0.38511246, -0.05246475],
       [-0.06580459,  0.02826248,  0.23930213, ...,  0.11102349,
        -0.3851116 , -0.05244821],
       [-0.06556657,  0.02735009,  0.2171814 , ...,  0.11353415,
        -0.38502502, -0.0526458 ]], dtype=float32)

array([[-0.06561741,  0.0275508 ,  0.22198111, ...,  0.11299524,
        -0.38506138, -0.05260655],
       [-0.06569718,  0.02785897,  0.22942495, ...,  0.11215296,
        -0.38509858, -0.05254164],
       [-0.06573445,  0.02800029,  0.23286769, ...,  0.11176081,
        -0.385108  , -0.05251   ],
       [-0.06568942,  0.02782934,  0.22870544, ...,  0.11223471,
        -0.38509616, -0.05254814]], dtype=float32)

array([[-0.06559324,  0.0274559 ,  0.21970677, ...,  0.11325098,
        -0.3850454 , -0.05262538],
       [-0.06577191,  0.02814086,  0.23631021, ...,  0.11136699,
        -0.38511208, -0.05247736],
       [-0.06566589,  0.027739  ,  0.22651641, ...,  0.11248302,
        -0.3850869 , -0.05256756],
       [-0.06571781,  0.02793735,  0.23133253, ...,  0.11193586,
        -0.38510448, -0.05252425]], dtype=float32)

array([[-0.06564257,  0.0276489 ,  0.22434081, ...,  0.11272905,
        -0.38507572, -0.0525865 ],
       [-0.06568698,  0.02781999,  0.2284786 , ...,  0.11226046,
        -0.38509533, -0.05255016],
       [-0.06566387,  0.02773128,  0.22632957, ...,  0.11250415,
        -0.38508606, -0.05256923],
       [-0.06555977,  0.02732293,  0.21653499, ...,  0.11360647,
        -0.38501945, -0.05265088]], dtype=float32)